In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision.transforms import ToTensor, Normalize, Compose

In [2]:
from scripts.architecture import MLP
from scripts.train import *
from scripts.plot_utils import plot_loss_accuracy, plotValAccuracy

In [3]:
print(torch.__version__)
print(np.__version__)

1.9.0
1.20.3


## Create Parity Data Iterator

In [4]:
transforms = Compose([
    ToTensor(),
    Normalize((0.1307,), (0.3081,))
])

In [5]:
# doesn't perform and transformation until we call the loader
trainset = torchvision.datasets.MNIST(root='data', train=True, download=True, transform=transforms)
testset = torchvision.datasets.MNIST(root='data', train=False, download=True, transform=transforms)

/home/john/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
learn_rate = 0.05
num_epochs = 20
batch_size = 128
loss_fn = torch.nn.BCEWithLogitsLoss()

### For k = 1

In [ ]:
k=1
model = MLP(k, "ReLU")
optimizer = torch.optim.Adadelta(model.parameters(), lr=learn_rate, weight_decay = 0.001)

trainLostList, trainAccList, valLossList, valAccList  = train_model(model, k, trainset, testset, loss_fn, optimizer, num_epochs, batch_size, validate_model = True,
                                                                     performance=accuracy_manual, device="cuda:0", lr_scheduler=None)


In [ ]:
plot_loss_accuracy(trainLostList,valLossList,trainAccList,valAccList,num_epochs)

In [8]:
k=1
model2 = MLP(k, "ReLU")
optimizer = torch.optim.SGD(model2.parameters(), lr=learn_rate)

trainLostList, trainAccList, valLossList, valAccList  = train_model(model2, k, trainset, testset, loss_fn, optimizer, num_epochs, batch_size, validate_model = True,
                                                                     performance=accuracy_manual, device="cpu",lr = learn_rate, lr_scheduler=None, updateWManually=False)


Training on cpu


/home/john/anaconda3/lib/python3.8/site-packages/torch/autograd/__init__.py:147: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/cuda/CUDAFunctions.cpp:115.)
  Variable._execution_engine.run_backward(


Epoch 1 completed. Loss - total: 11845.9674 - average: 0.1974; Performance: 0.9208
TESTING - loss 1181.6230304539204 - performance 0.9571
Epoch 2 completed. Loss - total: 5777.0455 - average: 0.0963; Performance: 0.9655
TESTING - loss 835.5479165911674 - performance 0.9702
Epoch 3 completed. Loss - total: 4419.5596 - average: 0.0737; Performance: 0.9742
TESTING - loss 693.2081878185272 - performance 0.9767
Epoch 4 completed. Loss - total: 3673.7891 - average: 0.0612; Performance: 0.9784
TESTING - loss 610.065970569849 - performance 0.9798
Epoch 5 completed. Loss - total: 3168.9639 - average: 0.0528; Performance: 0.9815
TESTING - loss 555.7761751115322 - performance 0.9819
Epoch 6 completed. Loss - total: 2789.5517 - average: 0.0465; Performance: 0.9837
TESTING - loss 516.1964055150747 - performance 0.9827
Epoch 7 completed. Loss - total: 2488.0638 - average: 0.0415; Performance: 0.9856
TESTING - loss 486.1013423651457 - performance 0.9832
Epoch 8 completed. Loss - total: 2234.5499 - av

In [10]:
with open('SGD_k1_pytorch.txt', 'w') as f:
    for item in valAccList:
        f.write("%s\n" % item)

In [ ]:
plot_loss_accuracy(trainLostList,valLossList,trainAccList,valAccList,num_epochs)

### For k = 3

In [ ]:
k = 3

model3 = MLP(k,"ReLU")
optimizer = torch.optim.Adadelta(model3.parameters(), lr=learn_rate, weight_decay = 0.001)

trainLostList3, trainAccList3, valLossList3, valAccList3  = train_model(model3, k, trainset, testset, loss_fn, optimizer, num_epochs, batch_size, validate_model = True,
                                                                     performance=accuracy_manual, device="cuda:0", lr_scheduler=None)


In [ ]:
plot_loss_accuracy(trainLostList3,valLossList3,trainAccList3,valAccList3,num_epochs)

In [11]:
k = 3
model4 = MLP(k, "ReLU")
optimizer = torch.optim.SGD(model4.parameters(), lr=learn_rate, weight_decay = 0.001)

trainLostList4, trainAccList4, valLossList4, valAccList4  = train_model(model4, k, trainset, testset, loss_fn, optimizer, num_epochs, batch_size, validate_model = True,
                                                                     performance=accuracy_manual, device="cpu", lr_scheduler=None)


Training on cpu
Epoch 1 completed. Loss - total: 41782.3516 - average: 0.6964; Performance: 0.4999
TESTING - loss 6949.679255485535 - performance 0.5028
Epoch 2 completed. Loss - total: 41615.4926 - average: 0.6936; Performance: 0.4986
TESTING - loss 6917.064130306244 - performance 0.4944
Epoch 3 completed. Loss - total: 41082.2017 - average: 0.6847; Performance: 0.5073
TESTING - loss 6713.841676712036 - performance 0.5459
Epoch 4 completed. Loss - total: 39043.5335 - average: 0.6507; Performance: 0.5532
TESTING - loss 6315.076053142548 - performance 0.5686
Epoch 5 completed. Loss - total: 36368.0561 - average: 0.6061; Performance: 0.6206
TESTING - loss 5871.085584163666 - performance 0.6534
Epoch 6 completed. Loss - total: 34835.6712 - average: 0.5806; Performance: 0.6539
TESTING - loss 5625.438928604126 - performance 0.6616
Epoch 7 completed. Loss - total: 33804.9682 - average: 0.5634; Performance: 0.6679
TESTING - loss 5547.36864566803 - performance 0.6939
Epoch 8 completed. Loss - 

In [12]:
with open('SGD_k3_pytorch.txt', 'w') as f:
    for item in valAccList4:
        f.write("%s\n" % item)

In [ ]:
plot_loss_accuracy(trainLostList4,valLossList4,trainAccList4,valAccList4,num_epochs)

In [ ]:
# Add Lazy methods
learn_rate = 0.05
K = 3
num_epochs = 20

fig = plt.figure()
for activation in ["ReLU", "NTK", "Gaussian features", "ReLU features", "linear features", "SGD"]:
    model = MLP(K, activation)

    if "features" in activation:
        # deactivate the first layer
        optimizer = torch.optim.Adadelta(model.layer2.parameters(), lr = learn_rate, weight_decay=0.001)
    elif "NTK" in activation:
        paramsToUpdate = list(model.layer1.parameters()) + list(model.layer2.parameters())
        optimizer = torch.optim.Adadelta(paramsToUpdate, lr = learn_rate, weight_decay=0.001)
    elif "SGD" in activation:
        optimizer = torch.optim.SGD(model.parameters(), lr = learn_rate, weight_decay=0.001)
    else:
        optimizer = torch.optim.Adadelta(model.parameters(), lr = learn_rate, weight_decay=0.001)

    print("Activation:",activation)

    trainLostList, trainAccList, valLossList, valAccList  = train_model(model, K, trainset, testset, loss_fn, optimizer, num_epochs, 
                                                                        batch_size, validate_model = True, performance=accuracy, 
                                                                        device="cuda:0", lr_scheduler=None)

    plotValAccuracy(valAccList,num_epochs, activation, K)

fig.savefig(str(K) + "valAccuracy.png")
plt.show()
dataset = MNISTParity(trainset, K, 128)
dataset.plotRandomData()

# just need to find good lr and weight_decay values for lazy methods to have more similar plots to paper
